In [16]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, r2_score

In [3]:
data = pd.read_csv("dataset_for_models.csv", index_col=0)

In [5]:
columns = data.columns
data = data[columns[1:len(columns)]]

In [6]:
data

,WeightInit,Height,DateDiff,ObesityDegree,Age,Gender,Prothrombin time,Bilirubin total,Creatinine,Total cholesterol,Glucose (syv.),Potassium,AST,ALT,WBC- Leukocytes,HGB- Hemoglobin,HCT-Hematocrit,Target,TargetWeight
0,100.0,176.0,32,1.0,64.156164,-1,12.7,8.90,112.0,7.40,5.60,5.00,36.0,51.0,3.90,140.0,41.70,0,100.0
1,100.0,172.5,358,1.0,66.958904,-1,21.2,7.00,123.0,3.10,9.30,5.70,21.0,37.0,8.20,155.0,43.40,0,100.0
2,86.0,162.0,307,1.0,63.602740,1,16.5,7.75,97.5,3.35,7.70,4.95,20.0,27.5,7.35,140.5,41.25,0,86.0
3,83.0,163.0,162,1.0,73.065753,1,11.8,8.50,72.0,3.60,6.10,4.20,19.0,18.0,6.50,126.0,39.10,-1,80.0
4,90.0,164.0,219,1.0,73.452055,1,13.5,12.90,62.0,4.40,6.10,4.30,14.0,11.0,5.70,130.0,36.50,-1,89.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,100.0,168.0,317,2.0,43.616438,1,13.7,9.30,60.0,3.70,5.70,4.70,17.0,17.0,8.80,136.8,43.70,1,102.0
561,98.0,174.0,233,1.0,61.063014,1,13.7,11.50,66.0,4.70,4.95,4.55,18.5,21.0,10.00,149.0,44.50,-1,92.0
562,116.0,182.0,326,1.0,16.342466,-1,13.7,13.70,72.0,5.70,4.20,4.40,20.0,25.0,7.60,146.0,42.10,-1,102.0
563,130.0,189.0,142,2.0,23.298630,-1,13.7,13.70,78.0,3.70,4.60,4.00,23.0,42.0,12.60,155.0,44.40,1,134.0


In [7]:
columns

Index(['Name', 'WeightInit', 'Height', 'DateDiff', 'ObesityDegree', 'Age',
       'Gender', 'Prothrombin time', 'Bilirubin total', 'Creatinine',
       'Total cholesterol', 'Glucose (syv.)', 'Potassium', 'AST', 'ALT',
       'WBC- Leukocytes', 'HGB- Hemoglobin', 'HCT-Hematocrit', 'Target',
       'TargetWeight'],
      dtype='object')

In [11]:
X = data[['WeightInit', 'Height', 'DateDiff', 'ObesityDegree', 'Age',
       'Gender', 'Prothrombin time', 'Bilirubin total', 'Creatinine',
       'Total cholesterol', 'Glucose (syv.)', 'Potassium', 'AST', 'ALT',
       'WBC- Leukocytes', 'HGB- Hemoglobin', 'HCT-Hematocrit']]
y = data["TargetWeight"]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

In [15]:
slr = LinearRegression()

slr.fit(X_train, y_train)
y_train_pred = slr.predict(X_train)
y_test_pred = slr.predict(X_test)

In [17]:
print('MSE train: {:.3f}, test: {:.3f}'.format(
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: {:.3f}, test: {:.3f}'.format(
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train: 31.489, test: 42.043
R^2 train: 0.877, test: 0.825
